# Dataset ANP
[gov.br: dados abertos](https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/acervo-de-dados-tecnicos)

Banco de dados:
1. Cruzamento de dados de geolocalização da ANP com dados de produtividade mensal
2. Seleção do do campo de Jubarte, na Bacia de Campos
3. Append nas tabelas de produção mensal


Considerações:
* Objeto de estudo: Bacia de Campos, campo de Jubarte
* Dados mensais ANP até dez23
* Produtividade, Horas Trabalhadas > 0

## Bibliotecas

In [13]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import sympy as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import feyn
from scipy.optimize import curve_fit


## Dataframe dados de Predição
Bacia de Campos

In [14]:
file_analise = r"C:\Users\guilh\OneDrive - UFRGS\Graduação\10-Trabalho de Conclusão do Curso\Predição de Poços de Petróleo\Pocos\analise.xlsx"
sheet_analise = 'campos'

df_campos = pd.read_excel(file_analise, sheet_name=sheet_analise)
df_campos.sort_values(by=['Campo', 'Periodo'], inplace=True)

df_campos.columns.tolist()


['Bacia',
 'Poco',
 'Campo',
 'Periodo',
 'Petroleo (bbl/dia)',
 'Tempo de producao (h/mes)',
 'Latitude',
 'Longitude',
 'Dummy_repeticao',
 'Days',
 'MesProducao',
 'Produtividade',
 'DeltaProdutividade']

## Seleção do campo analisado

In [15]:
campos_frequentes = df_campos['Campo'].value_counts().nlargest(5)
campos_frequentes


Campo
JUBARTE         1374
MARLIM LESTE     303
CARATINGA         66
BARRACUDA         42
MARLIM            41
Name: count, dtype: int64

In [16]:
df_campos.loc[df_campos['MesProducao'] < 0, 'MesProducao'] = 0


In [17]:
campo_analise = 'JUBARTE'
df_campo_analise = df_campos[df_campos['Campo'] == campo_analise]

df_campo_analise['Poco'].value_counts().nlargest(50)


Poco
7BFR7ESS         98
7JUB57DPAESS     97
7JUB58DPAESS     97
7BAZ8ESS         97
8JUB39ESS        93
7BFR12PAESS      91
6BRSA631DBESS    90
7BAZ4ESS         89
7JUB34HESS       87
6BRSA1222AESS    83
7BAZ3ESS         83
6BRSA639ESS      81
7JUB44ESS        70
7JUB55ESS        63
7BAZ9HAESS       43
7JUB45ESS        41
7BAZ6ESS         29
7BAZ10DAESS      27
8JUB59DESS       11
7JUB63ESS         3
7JUB36ESS         1
Name: count, dtype: int64

## Outliers

In [18]:
desvios = 2
mean = df_campo_analise['DeltaProdutividade'].mean()
stdev = df_campo_analise['DeltaProdutividade'].std()

lower_limit = mean - (desvios * stdev)
upper_limit = mean + (desvios * stdev)

df_campo_analise.shape[0]

1374

In [19]:
df_campo_analise = df_campo_analise[(df_campo_analise['DeltaProdutividade'] >= lower_limit) & (df_campo_analise['DeltaProdutividade'] <= upper_limit)]

df_campo_analise.shape[0]

1341

## Seleção de Variáveis

In [20]:
colunas_analise = [ 'Latitude',
                    'Longitude',
                    'MesProducao',
                    'Produtividade'
]

variavel_dependente = 'Produtividade'

variaveis_independentes = [ 'Latitude',
                            'Longitude', 
                            'MesProducao'
]

df_campo_analise = df_campo_analise[colunas_analise]
print(df_campo_analise.columns.tolist())

print('STDev:', df_campo_analise['Produtividade'].std())

['Latitude', 'Longitude', 'MesProducao', 'Produtividade']
STDev: 268.06631759184836


# Loop

In [21]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from feyn import QLattice
import feyn.tools
import pandas as pd

# Parâmetros do TensorFlow
l1_value = l2_value = 0.001
fator_dropout = 0.2
funcao_ativacao = 'relu'
neuronios_por_camada = 256
batches = 32

# Inicialização QLattice
ql = QLattice()

# Lista de resultados (inicialização vazia)
results_list = []

# Loop conjunto para todos os algoritmos
for i in range(200):
    # Scaling + Split do dataset (reformata o dataset a cada iteração do loop)
    scaler = StandardScaler()
    df_features_scaled = scaler.fit_transform(df_campo_analise[['Latitude', 'Longitude', 'MesProducao']])
    x_train, x_test, y_train, y_test = train_test_split(df_features_scaled, df_campo_analise['Produtividade'], train_size=0.70, test_size=0.30)
    
    # Rede Neural Artificial
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(3,)),
        tf.keras.layers.Dense(neuronios_por_camada, activation=funcao_ativacao, kernel_regularizer=tf.keras.regularizers.l2(l2=l2_value)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(fator_dropout),
        tf.keras.layers.Dense(neuronios_por_camada, activation=funcao_ativacao, kernel_regularizer=tf.keras.regularizers.l2(l2=l2_value)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(fator_dropout),
        tf.keras.layers.Dense(1)
    ])

    otimizador = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer=otimizador, loss='mean_squared_error')

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

    history = model.fit(x_train, y_train, 
              epochs=200,
              verbose=0,
              batch_size=batches,
              validation_data=(x_test, y_test),
              callbacks=[early_stopping])

    y_pred_test_tf = model.predict(x_test)

    test_r2_tf = r2_score(y_test, y_pred_test_tf)
    test_rmse_tf = root_mean_squared_error(y_test, y_pred_test_tf)
    test_mae_tf = mean_absolute_error(y_test, y_pred_test_tf)

    # Regressão Simbólica
    train_feyn = pd.DataFrame(x_train, columns=['Latitude', 'Longitude', 'MesProducao'])
    train_feyn['Produtividade'] = y_train.values

    test_feyn = pd.DataFrame(x_test, columns=['Latitude', 'Longitude', 'MesProducao'])
    test_feyn['Produtividade'] = y_test.values

    models = ql.auto_run(data=train_feyn, output_name='Produtividade')
    best = models[0]

    y_true_feyn = test_feyn['Produtividade'].values
    y_pred_feyn = best.predict(test_feyn)

    test_r2_feyn = r2_score(y_true_feyn, y_pred_feyn)
    test_rmse_feyn = root_mean_squared_error(y_true_feyn, y_pred_feyn)
    test_mae_feyn = mean_absolute_error(y_true_feyn, y_pred_feyn)

    # Regressão Linear
    modelo_linear = LinearRegression()
    modelo_linear.fit(x_train, y_train)

    y_test_pred_linear = modelo_linear.predict(x_test)

    test_r2_linear = r2_score(y_test, y_test_pred_linear)
    test_rmse_linear = root_mean_squared_error(y_test, y_test_pred_linear)
    test_mae_linear = mean_absolute_error(y_test, y_test_pred_linear)

    # Regressão Polinomial
    modelo_poli = Pipeline([
        ('poly', PolynomialFeatures(degree=2)),
        ('linear', LinearRegression())
    ])

    modelo_poli.fit(x_train, y_train)

    y_test_pred_poli = modelo_poli.predict(x_test)

    test_r2_poli = r2_score(y_test, y_test_pred_poli)
    test_rmse_poli = root_mean_squared_error(y_test, y_test_pred_poli)
    test_mae_poli = mean_absolute_error(y_test, y_test_pred_poli)

    # Armazenamento
    results_list.append({
        'n treinamento': i+1, 
        'r2_tensorflow': test_r2_tf, 
        'mae_tensorflow': test_mae_tf, 
        'rmse_tensorflow': test_rmse_tf,
        'r2_feyn': test_r2_feyn, 
        'mae_feyn': test_mae_feyn, 
        'rmse_feyn': test_rmse_feyn,
        'r2_linear': test_r2_linear, 
        'mae_linear': test_mae_linear, 
        'rmse_linear': test_rmse_linear,
        'r2_poli': test_r2_poli, 
        'mae_poli': test_mae_poli, 
        'rmse_poli': test_rmse_poli
    })
    print(f"Iteração {i+1} concluída")

unified_results = pd.DataFrame(results_list)
unified_results.to_excel(r'C:\Users\guilh\OneDrive - UFRGS\Graduação\10-Trabalho de Conclusão do Curso\Predição de Poços de Petróleo\unified_results.xlsx', index=False)
print("Resultados arquivados")


Iteração 200 concluída
Resultados arquivados
